In [1]:
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

# Warum auch immer, bekomme ich einen fehler wenn ich das hier nicht drinne habe. 
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
# Tensoren. Multidimensionale Arrays die schnell durch GPU schneller berechnet werden können (je nach lib)
# Tensoren sind unveränderlich
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(256))
print(tf.reduce_sum([123, 456, 789]))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(65536, shape=(), dtype=int32)
tf.Tensor(1368, shape=(), dtype=int32)


In [3]:
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("And NumPy operations convert Tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

TensorFlow operations convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
And NumPy operations convert Tensors to numpy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The .numpy() method explicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


In [4]:
# TensorFlow nutzt wenn möglich eine GPU. Ich hatte in letzterzeit immer Probleme damit, deshalb immer schauen ob die wirklich verwendet wird
x = tf.random.uniform([3, 3])

print("Is there a GPU available: "),
print(tf.config.experimental.list_physical_devices("GPU"))

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0')) # Tensor.device sagt einem immer wo der Tensor gespeichert ist

Is there a GPU available: 
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is the Tensor on GPU #0:  
True


In [9]:
# Tensorflow bestimmt selbst, welche Quelle am besten geeignet ist. Es ist aber möglich es selbst zu bestimmen mit tf.device:
import time

def time_matmul(x):
    start = time.time()
    for loop in range(100):
        tf.matmul(x, x)

    result = time.time()-start

    print("100 loops: {:0.2f}ms".format(1000*result))

# Force CPU
print("On CPU:")
with tf.device("CPU:0"):
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("CPU:0")
    time_matmul(x)

# Force GPU wenn möglich!
if tf.config.experimental.list_physical_devices("GPU"):
    print("On GPU:")
    with tf.device("GPU:0"):
        x = tf.random.uniform([1000, 1000])
        assert x.device.endswith("GPU:0")
        time_matmul(x)

On CPU:
100 loops: 564.02ms
On GPU:
100 loops: 1.99ms


In [10]:
# Datasets sind ein wesentlicher bestandteil und werden IMMER benötigt. Das richtige einlesen und verwenden ist daher wichtig:

# direkt aus memory einlesen
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# Erstellen einer CSV Datei die als beispiel Dataset dient. 
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
    f.write("""Line 1
        Line 2
        Line 3
        """)

ds_file = tf.data.TextLineDataset(filename)


In [11]:
# Die Tensorflow Datasets erlauben schneller und direktes bearbeiten der Daten...

ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

In [12]:
print('Elements of ds_tensors:')
for x in ds_tensors:
    print(x)

print('\nElements in ds_file:')
for x in ds_file:
    print(x)

Elements of ds_tensors:
tf.Tensor([1 4], shape=(2,), dtype=int32)
tf.Tensor([ 9 25], shape=(2,), dtype=int32)
tf.Tensor([16 36], shape=(2,), dtype=int32)

Elements in ds_file:
tf.Tensor([b'Line 1' b'        Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'        Line 3' b'        '], shape=(2,), dtype=string)


In [13]:
#############################################
### Erstellen von eigenen Layers ###########
############################################


print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [14]:
# TensorFlow hat komplett Keras als package dabei. 
# Bei Keras sind Layers Objecte mit dem ersten parameter als output dimension

layer = tf.keras.layers.Dense(100)

# Die Inputs sind meist unwichtig und können selbst bestimmt werden durch das Modell, können aber auch selbst angegeben werden:
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))


In [15]:
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [16]:
# Die Daten eines layers sind einfach zugänglich..
layer.variables, layer.kernel, layer.bias

([<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
  array([[-0.40929544,  0.3124389 ,  0.40546972, -0.09909713, -0.07109708,
           0.6319142 , -0.48159188,  0.05686063, -0.0063591 ,  0.06572825],
         [ 0.26137668, -0.0314014 ,  0.41012746, -0.03485429,  0.31970614,
          -0.41851783,  0.4239146 , -0.44705364,  0.02539951,  0.5565849 ],
         [-0.08241683,  0.17229319,  0.4672262 , -0.25725383, -0.43908307,
          -0.00500816, -0.39614558, -0.21510434, -0.50964665,  0.5362541 ],
         [-0.526144  , -0.45505708, -0.08405441,  0.62521404,  0.13114089,
           0.34899455,  0.4711463 , -0.5616564 , -0.5248999 , -0.34409043],
         [ 0.30135167, -0.484842  ,  0.1627689 ,  0.12434554,  0.25461906,
           0.4656729 , -0.5514828 ,  0.16638374, -0.10623926,  0.57282716]],
        dtype=float32)>,
  <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>],
 <tf.Variable 'den

In [17]:
# Erstellen eines eigenen Layers:
# Erben von der Layer Klasse und die Methoden: __init__, build und call implementieren
class NicoDense(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(NicoDense, self).__init__()
        self.num_outputs = num_outputs
        
    def build(self, input_shape):
        self.kernel = self.add_weight("kernel", shape=[int(input_shape[-1]), self.num_outputs])
        
    def call(self, input):
        return tf.matmul(input, self.kernel)
    
layer = NicoDense(10)

In [18]:
_ = layer(tf.zeros([10, 5]))
print([var.name for var in layer.trainable_variables])

['nico_dense/kernel:0']


In [19]:
# Composing Layers ist das A und O für bessere Modelle, TF bietet mit Keras einfache und gute Möglichkeiten:

class ResnetIdentityBlock(tf.keras.Model):
    def __init__(self, kernel_size, filters):
        super(ResnetIdentityBlock, self).__init__(name='')
        filters1, filters2, filters3 = filters

        self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
        self.bn2a = tf.keras.layers.BatchNormalization()

        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()

        self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
        self.bn2c = tf.keras.layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2c(x)
        x = self.bn2c(x, training=training)

        x += input_tensor
        return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])


In [20]:
_ = block(tf.zeros([1, 2, 3, 3]))
block.layers

In [21]:
block.summary()

Model: "resnet_identity_block"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  4         
_________________________________________________________________
batch_normalization (BatchNo multiple                  4         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_1 (Batch multiple                  8         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  9         
_________________________________________________________________
batch_normalization_2 (Batch multiple                  12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
________________________________________________

In [24]:
nico_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1), input_shape=(None, None, 3)),
                            tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])

nico_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [25]:
nico_seq.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, None, None, 1)     4         
_________________________________________________________________
batch_normalization_6 (Batch (None, None, None, 1)     4         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, None, None, 2)     4         
_________________________________________________________________
batch_normalization_7 (Batch (None, None, None, 2)     8         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, None, None, 3)     9         
_________________________________________________________________
batch_normalization_8 (Batch (None, None, None, 3)     12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
_________________________________________________________

In [26]:
session.close()